## Handling Missing Data in a pandas `DataFrame`
### Working with pandas
*Curtis Miller*

In this notebook I demonstrate approaches to handling missing data in a pandas `DataFrame`. The first thing I do is create a `DataFrame` `df` that contains missing data. (Because numbers are random, you should expect your results to differ.)

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import random

# Create a data frame of random numbers, some randomly censored
vals = np.random.randn(21)
vals[random.sample([i for i in range(21)], 5)] = np.nan
df = DataFrame(vals.reshape(7, 3), columns = ["AAA", "BBB", "CCC"])
df

,AAA,BBB,CCC
0,0.609616,-1.542960,-0.276952
1,-0.303875,NaN,2.083666
2,-0.982059,0.018547,0.374198
3,NaN,-0.003428,NaN
4,NaN,-0.246710,-0.616030
5,1.523133,0.560754,NaN
6,0.915636,-0.584181,0.038213


In [2]:
srs = Series([2, 3, 3, 9, 8, np.nan, 8, np.nan, 4, 4, 5])
print(srs)

0     2.0
1     3.0
2     3.0
3     9.0
4     8.0
5     NaN
6     8.0
7     NaN
8     4.0
9     4.0
10    5.0
dtype: float64


Here we see methods for detecting missing data. These methods produce identical (or 100% contradictory) results.

In [3]:
np.isnan(df)

,AAA,BBB,CCC
0,False,False,False
1,False,True,False
2,False,False,False
3,True,False,True
4,True,False,False
5,False,False,True
6,False,False,False


In [4]:
df.isnull()

,AAA,BBB,CCC
0,False,False,False
1,False,True,False
2,False,False,False
3,True,False,True
4,True,False,False
5,False,False,True
6,False,False,False


In [5]:
df.notnull()    # Opposite of isnull() and isnan()

,AAA,BBB,CCC
0,True,True,True
1,True,False,True
2,True,True,True
3,False,True,False
4,False,True,True
5,True,True,False
6,True,True,True


Here's what removing missing information looks like.

In [6]:
df.dropna()

,AAA,BBB,CCC
0,0.609616,-1.542960,-0.276952
2,-0.982059,0.018547,0.374198
6,0.915636,-0.584181,0.038213


In [7]:
print(srs.dropna())

0     2.0
1     3.0
2     3.0
3     9.0
4     8.0
6     8.0
8     4.0
9     4.0
10    5.0
dtype: float64


Now let's look at more interesting approaches to filling missing information.

In [8]:
xbar = srs.mean()    # By default, ignores nan
print(xbar)

5.111111111111111


In [9]:
print(srs.fillna(0))

0     2.0
1     3.0
2     3.0
3     9.0
4     8.0
5     0.0
6     8.0
7     0.0
8     4.0
9     4.0
10    5.0
dtype: float64


In [10]:
print(srs.fillna(xbar))

0     2.000000
1     3.000000
2     3.000000
3     9.000000
4     8.000000
5     5.111111
6     8.000000
7     5.111111
8     4.000000
9     4.000000
10    5.000000
dtype: float64


In [11]:
# How does the mean of this data compare to before?
srs.fillna(xbar).mean()

5.111111111111112

In [12]:
# What about the standard deviation (a measure of how dispersed data is)?
srs.std()

2.5712081034235856

In [13]:
srs.fillna(xbar).std()

2.2997584414213788

Filling missing data with the mean of that data is not cost-free; while the mean is preserved, other important metrics (such as the standard deviation) are affected, which may contaminate some algorithms (we made the data appear more concentrated than the original data was).

Here's a trick: replace the data with *randomly generated* data with the same mean and standard devation as the original data. We may pick random values from our data set and fill in missing data with those values. This resembles a statistical technique known as bootstrapping.

I demonstrate below.

In [14]:
s = srs.std()
# Generate a NumPy ndarray filled with randomly generated data, of the same length as the missing data
rep = Series(np.random.choice(srs[srs.notnull()], size=2), index=[5, 7])
print(rep)

5    3.0
7    4.0
dtype: float64


In [15]:
srs.fillna(rep)

0     2.0
1     3.0
2     3.0
3     9.0
4     8.0
5     3.0
6     8.0
7     4.0
8     4.0
9     4.0
10    5.0
dtype: float64

In [16]:
srs.fillna(rep).mean()

4.818181818181818

In [17]:
srs.fillna(rep).std()

2.400757456228422

While random, the mean and standard deviation of the filled-in data set are both close to that of the original data set. (Not that this approach is perfect either; why the Normal distribution?).

Now let's look at `df` again. Let's try to fill missing data.

In [18]:
df.fillna(0)

,AAA,BBB,CCC
0,0.609616,-1.542960,-0.276952
1,-0.303875,0.000000,2.083666
2,-0.982059,0.018547,0.374198
3,0.000000,-0.003428,0.000000
4,0.000000,-0.246710,-0.616030
5,1.523133,0.560754,0.000000
6,0.915636,-0.584181,0.038213


In [19]:
df.mean()

AAA    0.352490
BBB   -0.299663
CCC    0.320619
dtype: float64

In [20]:
df.fillna(df.mean())

,AAA,BBB,CCC
0,0.609616,-1.542960,-0.276952
1,-0.303875,-0.299663,2.083666
2,-0.982059,0.018547,0.374198
3,0.352490,-0.003428,0.320619
4,0.352490,-0.246710,-0.616030
5,1.523133,0.560754,0.320619
6,0.915636,-0.584181,0.038213


In [21]:
df.std()

AAA    0.995677
BBB    0.715384
CCC    1.051827
dtype: float64

In [22]:
df.fillna(df.mean()).std()    # All standard deviations go down

AAA    0.812967
BBB    0.653053
CCC    0.858813
dtype: float64

What does the "fill with random data" trick used above look like here?

In [23]:
col='AAA'
df[col][df[col].notnull()]

0    0.609616
1   -0.303875
2   -0.982059
5    1.523133
6    0.915636
Name: AAA, dtype: float64

In [24]:
# We will fill missing data via a dict
rep_df = {col: Series(np.random.choice(df[col][df[col].notnull()],    # Create a Series of random values from col...
                                       size=df.isnull()[col].value_counts()[True]),     # ... as many as there are missing values
                                                                                        # in col...
                      index=df[col][df[col].isnull()].index)    # ... and having an index corresponding to the missing values
                                                                # in the column col of df ...
          for col in df}    # ... for each column in df
rep_df

{'AAA': 3    0.915636
 4    0.609616
 dtype: float64, 'BBB': 1   -1.54296
 dtype: float64, 'CCC': 3    0.374198
 5    2.083666
 dtype: float64}

In [25]:
df.fillna(rep_df)

,AAA,BBB,CCC
0,0.609616,-1.542960,-0.276952
1,-0.303875,-1.542960,2.083666
2,-0.982059,0.018547,0.374198
3,0.915636,-0.003428,0.374198
4,0.609616,-0.246710,-0.616030
5,1.523133,0.560754,2.083666
6,0.915636,-0.584181,0.038213


In [26]:
df.fillna(rep_df).mean()

AAA    0.469672
BBB   -0.477277
CCC    0.580137
dtype: float64

In [27]:
df.fillna(rep_df).std()

AAA    0.841884
BBB    0.804553
CCC    1.085136
dtype: float64

As you encounter different problems you may come upon other solutions to filling in missing values. Here are some examples.

For numeric data:

* Fill in with a "neutral" value, like 0, 1, or sample mean
* Fill with taylored values to preserve select statistics (like the mean or standard deviation), randomly assigned to rows
* Fill with independently generated random numbers with same statistical properties as the data

For categorical data

* Fill with most common value
* Fill with values chosen with a frequency that would preserve observed frequencies, randomly assigned to rows
* Fill with independently generated random values chosen with the same frequency as the observed frequencies

None of this even covers imputation! There's many ways to fill missing values.